In [22]:
from src.lib.utils.FileHandling.FileHandlingInterface import *
from src.lib.probabilityEstim.Likelihood import *
import matplotlib.pyplot as plt
import logging
import time
import warnings
from src.lib.utils.FileHandling.FileHandlingInterface import * 

In [23]:
KEA_IMAGE_PATH = r".\data\github\kea.jpg"
KEA_SCRIBBLE_PATH = r".\data\github\kea_scribble.png"


In [24]:
sample_image_kea = TargetImage(KEA_IMAGE_PATH)
sample_scribble_kea = EncodedScribble(
    KEA_SCRIBBLE_PATH, 
    is_xml = 2
)

In [32]:
sample_image_kea.get_image_array().shape

(3, 640, 382)

In [25]:
sample_scribble_kea.get_encoded_scribble()[0].shape

(9213, 2)

In [26]:
PASCAL_DEMO_SCRIBBLE_PATH = r".\data\demoPascal\2008_003147.xml"
PASCAL_DEMO_SCRIBBLE_PATH1 = r".\data\demoPascal\2010_005258.xml"

In [28]:
n_classes = sample_scribble_kea.get_n_classes()

In [29]:
likelihood = Likelihood(
    n_classes,  
    return_ = 0,
    #alpha = 13, 
    #sigma = 18
)

In [30]:
time0 = time.time()
fitted_likelihood = likelihood.fit(
    sample_image_kea, 
    sample_scribble_kea,
    normalize = True,
    neg_log = True
)
print(f"{time.time() - time0}")

IndexError: index 455 is out of bounds for axis 2 with size 382